In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import time
import math
from tqdm import tqdm
from importlib import reload
from os.path import join, normpath
from bs4 import BeautifulSoup
import os
import roamability as rb
import re

downloads = 'C:/Users/balob/Downloads'

In [2]:
df = pd.read_csv(join(downloads, 'RBB - Sheet1 (49).csv'), dtype='object')
df.head(3)

,SerialNumber
0,8997219121000036000
1,8997219121000036001
2,8997219121000036002


In [3]:
sql_srt = """SELECT
r.RESELLER_NAME
,rpr.roaming_plan_name
,ipr.international_plan_name
,a.account_name
,rpa.roaming_plan_name
,ipa.international_plan_name
,ss.STATUS AS SUB_STATUS
,COUNT(*) AS TOT_SUB
FROM RESELLERS r
INNER JOIN ACCOUNTS ra ON r.ACCOUNT_ID = ra.ACCOUNT_ID
INNER JOIN ACCOUNTS a ON r.RESELLER_ID = a.RESELLER_ID
LEFT OUTER JOIN ROAMING_PLANS rpr ON ra.pricing_plan_id = rpr.roaming_plan_id
LEFT OUTER JOIN ROAMING_PLANS rpa ON a.pricing_plan_id = rpa.roaming_plan_id
LEFT OUTER JOIN INTERNATIONAL_PLANS ipr ON ra.int_pricing_plan_id = ipr.international_plan_id
LEFT OUTER JOIN INTERNATIONAL_PLANS ipa ON a.int_pricing_plan_id = ipa.international_plan_id
INNER JOIN SUBSCRIBERS sr ON sr.ACCOUNT_ID = a.ACCOUNT_ID
INNER JOIN SUBSCRIBER_STATUS ss ON ss.SUBSCRIBER_ID = sr.SUBSCRIBER_ID
INNER JOIN SUBSCRIBER_IMSIS si ON si.SUBSCRIBER_ID = sr.SUBSCRIBER_ID
WHERE
r.deleted is NULL
AND ss.END_DATE IS NULL
AND ss.status = 'Active'
AND si.ICCID IN (""" + str(df.SerialNumber.tolist())[1:-1] + """)
GROUP BY
r.RESELLER_NAME
,rpr.roaming_plan_name
,ipr.international_plan_name
,a.account_name
,rpa.roaming_plan_name
,ipa.international_plan_name
,ss.STATUS
ORDER BY r.RESELLER_NAME,a.account_name"""

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_imsi = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

df_imsi

,RESELLER_NAME,roaming_plan_name,international_plan_name,account_name,roaming_plan_name,international_plan_name,SUB_STATUS,TOT_SUB
0,STI - Telinta,Combined Charge 2019,Telinta Own Calls,G1S_Pinguin(All),None,Telinta Own Calls,Active,4150


In [4]:
sql_srt = """
SELECT RESELLER_NAME,account_name,SUBSTRING(IMSI,1,12) AS IMSI_RANGE,COUNT(*) AS NUM
,MIN(IMSI) AS MIN_IMSI,MAX(IMSI) AS MAX_IMSI
FROM
(
SELECT
r.RESELLER_NAME
,a.account_name
,ss.STATUS AS SUB_STATUS
,si.IMSI
FROM RESELLERS r
INNER JOIN ACCOUNTS a
ON r.RESELLER_ID = a.RESELLER_ID
INNER JOIN SUBSCRIBERS sr
ON sr.ACCOUNT_ID = a.ACCOUNT_ID
INNER JOIN SUBSCRIBER_STATUS ss
ON ss.SUBSCRIBER_ID = sr.SUBSCRIBER_ID
INNER JOIN SUBSCRIBER_IMSIS si
ON ss.SUBSCRIBER_ID = si.SUBSCRIBER_ID
WHERE
r.deleted is NULL
AND ss.END_DATE IS NULL
--AND ss.status = 'Active'
AND si.ICCID IN (""" + str(df.SerialNumber.tolist())[1:-1] + """)
) t
GROUP BY RESELLER_NAME,account_name,SUBSTRING(IMSI,1,12)
ORDER BY SUBSTRING(IMSI,1,12),account_name"""
    
with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_imsi_ranges = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

df_imsi_ranges

,RESELLER_NAME,account_name,IMSI_RANGE,NUM,MIN_IMSI,MAX_IMSI
0,STI - Telinta,G1S_Pinguin(All),425019613036,500,425019613036500,425019613036999
1,STI - Telinta,G1S_Pinguin(All),425019613037,1000,425019613037000,425019613037999
2,STI - Telinta,G1S_Pinguin(All),425019613038,1000,425019613038000,425019613038999
3,STI - Telinta,G1S_Pinguin(All),425019613039,1000,425019613039000,425019613039999
4,STI - Telinta,G1S_Pinguin(All),425019613040,500,425019613040000,425019613040499
5,STI - Telinta,G1S_Pinguin(All),425019613044,150,425019613044000,425019613044149


In [5]:
df_imsi_ranges.apply(lambda x: int(x.MAX_IMSI) - int(x.MIN_IMSI), axis=1)

0    499
1    999
2    999
3    999
4    499
5    149
dtype: int64

In [6]:
from os.path import join
import pandas as pd

downloads = 'C:/Users/balob/Downloads'
source_file_name = 'STI_COMB_SPONSOR.csv'
result_file_name = 'toc_sponsor.rcjson'

sponsor_dict = {'Partner_STI_Ping':'Partner_STI_Comb','P4_STI_Ping':'P4_STI_Comb'}

df_dmi_imsi = pd.read_csv(join(downloads,source_file_name),dtype='str')
display(df_dmi_imsi.groupby('NAME',as_index=False)['IMSI'].count())
df_dmi_imsi.NAME.replace(to_replace=sponsor_dict,inplace=True)
display(df_dmi_imsi.groupby('NAME',as_index=False)['IMSI'].count())
display(df_dmi_imsi.tail())

ouf=open(join(downloads,result_file_name),'w')
#ouf.write('Some text\n')
#ouf.write(str(25))

num_val = df_dmi_imsi.count()[0]-1

t1='{"params":{"cli":"'
t2='","sponsor":"'
t3='","subscriber":"'
t4='","profileId":"'
t5='","imsi":"'
t6='","flagConfirmed":"0"},"type":"modify","caption":"dmi_s_imsi","objectId":"DMI.dmi_s_imsi","serviceId":"DMI"}'

ouf.write('[')
for i,[cli,imsi,pid,simsi,sponsor] in enumerate(df_dmi_imsi[['CLI_MSISDN','IMSI','PROF_ID','IMSI_NUMBER','NAME']].values):
    ouf.write(t1+cli+t2+sponsor+t3+imsi+t4+pid+t5+simsi+t6)
    if i < num_val:
        ouf.write(',')
ouf.write(']')

ouf.close()

,NAME,IMSI
0,P4_STI_Comb,5
1,P4_STI_Ping,4145
2,Partner_STI_Comb,5
3,Partner_STI_Ping,4145


,NAME,IMSI
0,P4_STI_Comb,4150
1,Partner_STI_Comb,4150


,CLI_MSISDN,IMSI,PROF_ID,IMSI_NUMBER,FLAG_CONFIRMED,NAME
8295,48791197647,425019613044147,2,260060149042647,NaN,P4_STI_Comb
8296,972540937648,425019613044148,1,425019613044148,NaN,Partner_STI_Comb
8297,48791197648,425019613044148,2,260060149042648,NaN,P4_STI_Comb
8298,972540937649,425019613044149,1,425019613044149,NaN,Partner_STI_Comb
8299,48791197649,425019613044149,2,260060149042649,NaN,P4_STI_Comb
